Installing gym: part of the open AI project. 

In [20]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


env = gym.make('CartPole-v1',render_mode="single_rgb_array").unwrapped # Loads the environment for open ai gym

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Replay Memory

- Replay memory stores the transitions that the agent observes allowing us to reuse this data later. By Sampling it randomly the transitions that build up a batch are decorrelated. It has been shown that this greatly stabilizes and improves the DQN training procedure.
- We're using replay memory. The Replay memory holds a transition between two states, which is what we're using to reward our ai; our reinforcement learning. In essence we're deciding what to do next based on any given scenario, like moving for checkmate if you're able to on a chess board.
- For example, if we wanted to know what to do in any given scenario, but we didn't want to just practice the same scenario over an over again, we'd use something like replay memory and select a random event that already happened and try to figure out what we'd do better or keep the same.
- Kinda like chess puzzles on chess.com, this is a way of improving gradually. Or like repeating a tasks and reviewing the result over and over again.
- Granted it's important that it is random as you can easily get stuck doing the same thing and not really improving at all. This is the best way to do things if the result is certain, who knows if the result is uncertain.

In [21]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward')) # Is used like a class in this example.


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

### DQN (Deep Q-Network) algorithm.

- The main Idea behind Q-learning is that if we had a function (Q∗ : State×Action -> R), that could tell us what our return would be, if we were to take an action in a given state, then we could easily construct a policy that maximizes our rewards.
- In other words, if we are able to create a function using our current state and action as parameters and our reward as a real number result, we would be able to quantify and thus maximize our rewards.
    - in simpler words, If we can put a number on an action, we can try and shoot for a higher number.
    - The only issues is that we can only guess that number, something NN's are effective at.
- One example of a policy that we can use to maximize our rewards is `π ∗(s)=argmax(sub(a)) Q∗(s,a)`
- This can apply to more than just neural networks, for example if we wanted to guess which food we'd like at a vending machine, we could quantify our answers and use the result as a way of guaging if our answers are truly the best.
- I'm not aware of any alternatives, so I cannot critique this.


### Bellman equation

- The Bellman equation asserts that a long term rewards is equal to the rewards from the current action combined with the expected rewards from future actions.
- In other words, We can decide rewards based on the current state and all future actions based on that current state.
- For example, if we are trying to navigate a robot through a room, we can use the Bellman equation to decide a way to navigate through that room, based on the values we assign to each square ![For example](https://media.geeksforgeeks.org/wp-content/uploads/20210914210519/env21.png) (Credit to Geeksforgeeks)
- The is very useful when trying to decide (on a base level) which state is the most optimal state in any given context.


### Our NN

- Our Neural network will be a convolutional neural network that takes the current and previous screen patches and outputs Q(s,left) and Q(s,right) where s is the input of the network.
- In other words, our NN tries to predict the expected return of the outcome.


In [22]:
class DQN(nn.Module):
    def __init__(self,h,w,outputs):
        super(DQN,self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

> convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
>
> convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))

In [23]:
resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=Image.CUBIC),
                    T.ToTensor()])


def get_cart_location(screen_width):
    world_width = env.x_threshold * 2
    scale = screen_width / world_width
    return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART

def get_screen():
    # Returned screen requested by gym is 400x600x3, but is sometimes larger
    # such as 800x1200x3. Transpose it into torch order (CHW).
    screen = env.render().transpose((2, 0, 1))
    # Cart is in the lower half, so strip off the top and bottom of the screen
    _, screen_height, screen_width = screen.shape
    screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]
    view_width = int(screen_width * 0.6)
    cart_location = get_cart_location(screen_width)
    if cart_location < view_width // 2:
        slice_range = slice(view_width)
    elif cart_location > (screen_width - view_width // 2):
        slice_range = slice(-view_width, None)
    else:
        slice_range = slice(cart_location - view_width // 2,
                            cart_location + view_width // 2)
    # Strip off the edges, so that we have a square image centered on a cart
    screen = screen[:, :, slice_range]
    # Convert to float, rescale, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    return resize(screen).unsqueeze(0)


env.reset()
plt.figure()
plt.imshow(get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(),
           interpolation='none')
plt.title('Example extracted screen')
plt.show()

SyntaxError: invalid syntax (72233638.py, line 16)

w

In [17]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
init_screen = get_screen()
_, _, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
n_actions = env.action_space.n

policy_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

AttributeError: 'NoneType' object has no attribute 'transpose'

In [18]:
num_episodes = 50
for i_episode in range(num_episodes):
    # Initialize the environment and state
    env.reset()
    last_screen = get_screen()
    current_screen = get_screen()
    state = current_screen - last_screen
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)

        # Observe new state
        last_screen = current_screen
        current_screen = get_screen()
        if not done:
            next_state = current_screen - last_screen
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

AttributeError: 'NoneType' object has no attribute 'transpose'